In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import functions_mem
import warnings
warnings.filterwarnings("ignore")


# Load data 

In [2]:
df = pd.read_parquet('/Users/julianeoliveira/Downloads/aesop_2025_09_26_mun.parquet')

In [7]:
df[['co_ibge',
 'epiyear',
 'epiweek',
 'atend_totais',
 'atend_ivas',
 'atend_arbov',
 'atend_diarreicas',
 'atend_0a11',
 'year_week',
 'epidemi_cal_start',
 'epidemi_cal_end',
 'year_week_ts',
 'co_uf',
 'nm_uf',
 'co_ibge7',
 'nm_municipio',
 'sigla_uf',
 'co_reg_saude',
 'nm_reg_saude',]].to_parquet('/Users/julianeoliveira/Downloads/aesop_2025_09_26.parquet')

In [16]:
df = df[df.epiyear >= 2022]

#  Baseline and Epidemic threshold

In [17]:
lst_sea = [2022, 2023, 2024] #2020, 2021

results = []

for cod in df.co_ibge.unique():
    lst = ['co_ibge', 'epiyear', 'epiweek','year_week', 'atend_totais', 'atend_ivas']

    set_muni = df[df.co_ibge == cod][lst]

    baseline,  post_baseline, epidemic_threshold, post_threshold, df_thresholds_intensity = functions_mem.baseline_thresholds(set_muni, lst_sea)
    
    summary, details = functions_mem.mem_epidemic_period(set_muni, delta=0.02)
    
    summary = summary.assign(co_ibge = cod,
              low_level = df_thresholds_intensity.value.iloc[0],
              medium_level = df_thresholds_intensity.value.iloc[1],
              high_level = df_thresholds_intensity.value.iloc[2],
              baseline = baseline,
              epidemic_threshold = epidemic_threshold)
    
    results.append(summary)

In [18]:
final = pd.concat(results)

In [19]:
final

,epiyear,r_j_estr,k_start,k_end,co_ibge,low_level,medium_level,high_level,baseline,epidemic_threshold
0,2022,9,44,52,110001,49.690259,76.012041,85.746635,24.033333,68.331740
1,2023,16,13,28,110001,49.690259,76.012041,85.746635,24.033333,68.331740
2,2024,19,3,21,110001,49.690259,76.012041,85.746635,24.033333,68.331740
3,2025,28,2,29,110001,49.690259,76.012041,85.746635,24.033333,68.331740
0,2022,0,1,0,110002,106.908106,127.896472,134.563530,60.000000,87.704546
...,...,...,...,...,...,...,...,...,...,...
3,2025,20,12,31,522230,104.831151,158.944304,178.848948,55.956522,103.178122
0,2022,9,1,9,530010,17312.329523,28386.257340,32657.844813,8254.722222,17825.423771
1,2023,16,9,24,530010,17312.329523,28386.257340,32657.844813,8254.722222,17825.423771
2,2024,16,8,23,530010,17312.329523,28386.257340,32657.844813,8254.722222,17825.423771


In [20]:
final.to_parquet('/Users/julianeoliveira/Documents/Projects/AESOP/Artigo Classification/results/mem_output_29_09_25.parquet')